# Training of the Hierarchical model for miniwob++

In [ ]:
# uninstall transformers twice to be sure
!pip uninstall transformers
!pip uninstall transformers

In [ ]:
! pip install datasets transformers rouge-score nltk evaluate accelerate

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 468.7/468.7 kB 8.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 73.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.4/81.4 kB 6.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 215.3/215.3 kB 20.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 41.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.1/200.1 kB 16.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.2/212.2 kB 13.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 9.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.9/132.9 kB 10.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 91.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 264.6/264.6 k

In [ ]:
!pip install cloud-tpu-client==0.10 torch==2.0.0 torchvision==0.15.1 https://storage.googleapis.com/tpu-pytorch/wheels/colab/torch_xla-2.0-cp39-cp39-linux_x86_64.whl

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.7/115.7 MB 11.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.7/57.7 kB 3.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 120.2/120.2 kB 6.7 MB/s eta 0:00:00
  Attempting uninstall: uritemplate
    Found existing installation: uritemplate 4.1.1
    Uninstalling uritemplate-4.1.1:
      Successfully uninstalled uritemplate-4.1.1
  Attempting uninstall: google-api-core
    Found existing installation: google-api-core 2.11.0
    Uninstalling google-api-core-2.11.0:
      Successfully uninstalled google-api-core-2.11.0
  Attempting uninstall: google-api-python-client
    Found existing installation: google-api-python-client 2.84.0
    Uninstalling google-api-python-client-2.84.0:
      Successfully uninstalled google-api-python-client-2.84.0
ERROR: pip's dependency resolver does not currently take into account

In [ ]:
# TPU
import torch_xla.core.xla_model as xm
import os
assert os.environ['COLAB_TPU_ADDR'], 'Make sure to select TPU from Edit > Notebook settings > Hardware accelerator'


In [ ]:
from huggingface_hub import notebook_login

notebook_login()

Token is valid.
Your token has been saved in your configured git credential helpers (store).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [ ]:
from datasets import load_dataset

# Load dataset regarding action-step sequences
raw_datasets_actions = load_dataset("LucasThil/miniwob_plusplus_hierarchical_training_actions")

# Load dataset regarding task of hierarchical planning
raw_datasets_planning = load_dataset("LucasThil/miniwob_plusplus_hierarchical_planning")

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/42097 [00:00<?, ? examples/s]

Dataset parquet downloaded and prepared to /root/.cache/huggingface/datasets/LucasThil___parquet/LucasThil--miniwob_plusplus_hierarchical_training_actions-49f035c84c4bf8ff/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/10960 [00:00<?, ? examples/s]

Dataset parquet downloaded and prepared to /root/.cache/huggingface/datasets/LucasThil___parquet/LucasThil--miniwob_plusplus_hierarchical_planning-55282b447acace9c/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

In [ ]:
raw_datasets_actions, raw_datasets_planning

(DatasetDict({
     train: Dataset({
         features: ['history_episodes', 'instruction', 'actions', 'refs', 'keydown_text', 'subtask_completion'],
         num_rows: 42097
     })
 }),
 DatasetDict({
     train: Dataset({
         features: ['hierarchical_plans', 'standolone_instruction'],
         num_rows: 10960
     })
 }))

In [ ]:
# Add extra column regarding condensed target_labels
# We append together the (action, ref, text)
# Where action is either of type 'click' or 'type'
# ref is the targeted element number to perform the action on
# and type is the text content in case the action is 'type'
def condensed_targets(dataset, label):
  condensed_targets = []
  for row in dataset[label]:
    target_text = row['keydown_text']
    if target_text is None:
      target_text = ""
    target = '{' + row['actions'] + ', ' + str(row['refs']) + ', ' + target_text + ', ' + row['subtask_completion'] + '}'
    condensed_targets.append(target)
  return condensed_targets

raw_datasets_actions = raw_datasets_actions.map(lambda example: {k: v if v == v else "" for k, v in example.items()})
raw_datasets_actions['train'] = raw_datasets_actions['train'].add_column("targets", condensed_targets(raw_datasets_actions, 'train'))
raw_datasets_actions['train'] = raw_datasets_actions['train'].remove_columns(['actions', 'refs', 'keydown_text', 'instruction', 'subtask_completion'])
raw_datasets_actions

Map:   0%|          | 0/42097 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['history_episodes', 'targets'],
        num_rows: 42097
    })
})

In [ ]:
# Format the hierchical planning dataset, later we'll marge it in the action dataset
# First add the Command specifying that this row is about asking the Model to Devise a plan
raw_datasets_planning = raw_datasets_planning.map(lambda row: {'standolone_instruction': 'Devise a plan for the following instruction: ' + row['standolone_instruction']})  
raw_datasets_planning = raw_datasets_planning.map(lambda example: {k: v if v == v else "" for k, v in example.items()})

# Rename the columns
raw_datasets_planning = raw_datasets_planning.rename_column('standolone_instruction', 'history_episodes')
raw_datasets_planning = raw_datasets_planning.rename_column('hierarchical_plans', 'targets')
raw_datasets_planning

Map:   0%|          | 0/10960 [00:00<?, ? examples/s]

Map:   0%|          | 0/10960 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['targets', 'history_episodes'],
        num_rows: 10960
    })
})

In [ ]:
from datasets import Dataset
from datasets import concatenate_datasets, DatasetDict

# select how much of the datset we want to use
# shuffle the elements, and split for a test set
def reduce_split_dataset(raw_datasets, coef):
  raw_datasets['train'] = raw_datasets['train'].shuffle()
  raw_datasets['train'] = raw_datasets["train"].select(range(int(len(raw_datasets["train"])*coef)))
  
  raw_datasets = raw_datasets["train"].train_test_split(train_size=0.95, seed=20)
  val_dataset = raw_datasets.pop("test")
  raw_datasets = raw_datasets["train"].train_test_split(train_size=0.95, seed=20)
  raw_datasets.update({'validation': val_dataset})

  return raw_datasets
  

In [ ]:
raw_datasets_actions = reduce_split_dataset(raw_datasets_actions, 1)
raw_datasets_planning = reduce_split_dataset(raw_datasets_planning, 1)
raw_datasets_actions, raw_datasets_planning

(DatasetDict({
     train: Dataset({
         features: ['history_episodes', 'targets'],
         num_rows: 37992
     })
     test: Dataset({
         features: ['history_episodes', 'targets'],
         num_rows: 2000
     })
     validation: Dataset({
         features: ['history_episodes', 'targets'],
         num_rows: 2105
     })
 }),
 DatasetDict({
     train: Dataset({
         features: ['targets', 'history_episodes'],
         num_rows: 9891
     })
     test: Dataset({
         features: ['targets', 'history_episodes'],
         num_rows: 521
     })
     validation: Dataset({
         features: ['targets', 'history_episodes'],
         num_rows: 548
     })
 }))

In [ ]:

# Now combine both datasets in one
train_dataset = concatenate_datasets([raw_datasets_actions['train'], raw_datasets_planning['train']])
test_dataset = concatenate_datasets([raw_datasets_actions['test'], raw_datasets_planning['test']])
validate_dataset = concatenate_datasets([raw_datasets_actions['validation'], raw_datasets_planning['validation']])
dataset = DatasetDict({
    'train': train_dataset,
    'test': test_dataset,
    'validation': validate_dataset
})
dataset

DatasetDict({
    train: Dataset({
        features: ['history_episodes', 'targets'],
        num_rows: 47883
    })
    test: Dataset({
        features: ['history_episodes', 'targets'],
        num_rows: 2521
    })
    validation: Dataset({
        features: ['history_episodes', 'targets'],
        num_rows: 2653
    })
})

In [ ]:
# Explore the dataset to ensure there are no NaNs, or lengths too big

# Show NaNs
nan_examples = dataset['train'].filter(lambda example: any(val != val for val in example.values()))

# Print the NaN examples
for example in nan_examples:
    print(f"Example ID: {example['id']}")
    for key, value in example.items():
        if value != value:  # Check for NaN values
            print(f"{key}: {value}")
    print('\n')

short_examples = dataset['train'].filter(lambda example: any(len(str(val)) < 5 for val in example.values()))

# Print the short examples
for example in short_examples:
    print(f"Example ID: {example}")
    for key, value in example.items():
        if len(str(value)) < 5:  # Check for short string values
            print(f"{key}: {value}")
    print('\n')
    break

Filter:   0%|          | 0/47883 [00:00<?, ? examples/s]

Filter:   0%|          | 0/47883 [00:00<?, ? examples/s]

In [ ]:
import datasets
import random
import pandas as pd
from IPython.display import display, HTML

def show_random_elements(dataset, num_examples=10):
    assert num_examples <= len(dataset), "Can't pick more elements than there are in the dataset."
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset)-1)
        while pick in picks:
            pick = random.randint(0, len(dataset)-1)
        picks.append(pick)
    
    df = pd.DataFrame(dataset[picks])
    for column, typ in dataset.features.items():
        if isinstance(typ, datasets.ClassLabel):
            df[column] = df[column].transform(lambda i: typ.names[i])
    display(HTML(df.to_html()))

In [ ]:
show_random_elements(dataset["train"])

# Prepare Model

In [ ]:
from transformers import AutoTokenizer

checkpoint = "t5-base"
#checkpoint = 'LucasThil/T5_base_hierarchy1'
#checkpoint = "LucasThil/T5_base_hierarchy5"
#checkpoint = 'LucasThil/T5_base_hierarchy8_256'
checkpoint = 'LucasThil/T5_base_hierarchy17_256_512'
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

In [ ]:
tokenizer

T5TokenizerFast(name_or_path='LucasThil/T5_base_hierarchy17_256_512', vocab_size=32100, model_max_length=512, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'eos_token': '</s>', 'unk_token': '<unk>', 'pad_token': '<pad>', 'additional_special_tokens': ['<extra_id_0>', '<extra_id_1>', '<extra_id_2>', '<extra_id_3>', '<extra_id_4>', '<extra_id_5>', '<extra_id_6>', '<extra_id_7>', '<extra_id_8>', '<extra_id_9>', '<extra_id_10>', '<extra_id_11>', '<extra_id_12>', '<extra_id_13>', '<extra_id_14>', '<extra_id_15>', '<extra_id_16>', '<extra_id_17>', '<extra_id_18>', '<extra_id_19>', '<extra_id_20>', '<extra_id_21>', '<extra_id_22>', '<extra_id_23>', '<extra_id_24>', '<extra_id_25>', '<extra_id_26>', '<extra_id_27>', '<extra_id_28>', '<extra_id_29>', '<extra_id_30>', '<extra_id_31>', '<extra_id_32>', '<extra_id_33>', '<extra_id_34>', '<extra_id_35>', '<extra_id_36>', '<extra_id_37>', '<extra_id_38>', '<extra_id_39>', '<extra_id_40>', '<extra_id_41>', '<extra_id_42>

In [ ]:
max_input_length = 512
max_target_length = 256

def preprocess_function(examples):
    inputs = [doc for doc in examples["history_episodes"]]
    model_inputs = tokenizer(inputs, max_length=max_input_length, truncation=True, padding=True)

    # Setup the tokenizer for targets
    #with tokenizer.as_target_tokenizer():
    labels = tokenizer(examples["targets"], max_length=max_target_length, truncation=True, padding=True)

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

In [ ]:
preprocess_function(dataset['train'][:2])

{'input_ids': [[6185, 209, 210, 9138, 3, 2, 6965, 6273, 17592, 1755, 121, 3155, 2, 8481, 3, 23, 26, 17592, 210, 5846, 121, 6273, 17592, 1714, 121, 3155, 2, 8481, 3, 23, 26, 17592, 498, 121, 6273, 17592, 2534, 121, 3155, 2, 8481, 3, 23, 26, 17592, 2689, 15, 7, 18, 17068, 121, 6273, 17592, 519, 121, 3155, 2, 40, 10333, 6273, 17592, 2688, 121, 3155, 2, 77, 2562, 686, 17592, 10031, 2689, 121, 3, 23, 26, 17592, 524, 632, 121, 6273, 17592, 2658, 121, 701, 17592, 2666, 15, 121, 3155, 2, 87, 77, 2562, 3155, 2, 17, 6273, 17592, 10794, 536, 121, 1499, 17592, 15, 591, 15, 536, 32, 357, 26, 121, 3155, 2, 87, 17, 3155, 2, 87, 40, 10333, 3155, 2, 40, 10333, 6273, 17592, 20364, 3155, 2, 77, 2562, 686, 17592, 10031, 2689, 121, 3, 23, 26, 17592, 524, 536, 121, 6273, 17592, 2606, 121, 701, 17592, 89, 5405, 15, 121, 3155, 2, 87, 77, 2562, 3155, 2, 17, 6273, 17592, 18, 27156, 121, 1499, 17592, 940, 172, 208, 121, 3155, 2, 87, 17, 3155, 2, 87, 40, 10333, 3155, 2, 40, 10333, 6273, 17592, 536, 121, 3155, 2, 

In [ ]:
tokenized_datasets = dataset.map(preprocess_function, batched=True)

Map:   0%|          | 0/47883 [00:00<?, ? examples/s]

Map:   0%|          | 0/2521 [00:00<?, ? examples/s]

Map:   0%|          | 0/2653 [00:00<?, ? examples/s]

In [ ]:
print(len(tokenized_datasets['train'][0]['input_ids']), len(tokenized_datasets['train'][1]['input_ids']))


512 512


In [ ]:
from transformers import AutoModelForSeq2SeqLM, DataCollatorForSeq2Seq, Seq2SeqTrainingArguments, Seq2SeqTrainer

#model = AutoModelForSeq2SeqLM.from_pretrained(model_checkpoint)

# new:
data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=checkpoint)

In [ ]:
import evaluate
# Use ROUGE as a metric
rouge = evaluate.load("rouge")

In [ ]:
import numpy as np

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True, clean_up_tokenization_spaces=True) # added clean_up_tokenization_spaces

    result = rouge.compute(predictions=decoded_preds, references=decoded_labels, use_stemmer=True)

    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in predictions]
    result["gen_len"] = np.mean(prediction_lens)

    return {k: round(v, 4) for k, v in result.items()}

In [ ]:
from transformers import AutoModelForSeq2SeqLM, Seq2SeqTrainingArguments, Seq2SeqTrainer

model = AutoModelForSeq2SeqLM.from_pretrained(checkpoint)
model.config.ctc_zero_infinity = True

In [ ]:
trainer = ""
import gc
import torch
torch.cuda.empty_cache()
gc.collect()

50

In [ ]:
!pip install pytorch-lightning

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 718.6/718.6 kB 12.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.2/519.2 kB 37.2 MB/s eta 0:00:00


In [ ]:
training_args = Seq2SeqTrainingArguments(
    output_dir="T5_base_hierarchy18_256_512",
    evaluation_strategy="epoch",
    learning_rate=1e-7,
    #per_device_train_batch_size=16,
    #per_device_eval_batch_size=16,
    auto_find_batch_size=True,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=3,
    predict_with_generate=True,
    fp16=False,
    #fp16_full_eval=True,
    push_to_hub=True,
    #logging_nan_inf_filter=True,
    dataloader_drop_last=True,
    #optim='adamw_torch'

    # TPU part
    #tpu_num_cores=xm.xrt_world_size(),
    warmup_steps=100,
)

trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

trainer.train()

Cloning https://huggingface.co/LucasThil/T5_base_hierarchy18_256_512 into local empty directory.
/usr/local/lib/python3.9/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
You're using a T5TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss,Rouge1,Rouge2,Rougel,Rougelsum,Gen Len
1,0.034000,0.037584,0.851300,0.659400,0.851200,0.850900,12.171500
2,0.031800,0.037530,0.851200,0.658700,0.851000,0.850800,12.171100
3,0.031200,0.037512,0.851300,0.659200,0.851200,0.850900,12.171100


TrainOutput(global_step=17955, training_loss=0.03257509978891116, metrics={'train_runtime': 7280.5705, 'train_samples_per_second': 19.73, 'train_steps_per_second': 2.466, 'total_flos': 8.74707114000384e+16, 'train_loss': 0.03257509978891116, 'epoch': 3.0})

In [ ]:
trainer.push_to_hub()

Upload file pytorch_model.bin:   0%|          | 1.00/850M [00:00<?, ?B/s]

Upload file runs/Apr20_10-43-34_235d4dfb79ff/events.out.tfevents.1681987419.235d4dfb79ff.822.0:   0%|         …

To https://huggingface.co/LucasThil/T5_base_hierarchy18_256_512
   95af286..4889fe0  main -> main

   95af286..4889fe0  main -> main

To https://huggingface.co/LucasThil/T5_base_hierarchy18_256_512
   4889fe0..c56a134  main -> main

   4889fe0..c56a134  main -> main



'https://huggingface.co/LucasThil/T5_base_hierarchy18_256_512/commit/4889fe0c8cef1dddb26b2dbb738996e8d4c8ebf2'

In [ ]:
trainer.save_model("LucasThil/T5_base_hierarchy3")
trainer.save_config("LucasThil/T5_base_hierarchy3")

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <cell line: 2>:2                                                                              │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
AttributeError: 'Seq2SeqTrainer' object has no attribute 'save_config'

In [ ]:
!transformers-cli upload LucasThil/T5_base_hierarchy3

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <cell line: 1>:1                                                                              │
│                                                                                                  │
│ /usr/local/lib/python3.9/dist-packages/google/colab/_shell.py:98 in system                       │
│                                                                                                  │
│    95 │   if pip_warn:                                                                           │
│    96 │     kwargs.update({'also_return_output': True})                                          │
│    97 │                                                                                          │
│ ❱  98 │   output = _system_commands._system_compat(self, *args, **kwargs)  # pylint:disable=pr   │
│    99 │                                                                                          │
│   100 │   if pip_warn:                                                                           │
│   101 │     _pip.print_previous_import_warning(output)                                           │
│                                                                                                  │
│ /usr/local/lib/python3.9/dist-packages/google/colab/_system_commands.py:453 in _system_compat    │
│                                                                                                  │
│   450   # We set a higher depth than the IPython system command since a shell object             │
│   451   # is expected to call this function, thus adding one level of nesting to the             │
│   452   # stack.                                                                                 │
│ ❱ 453   result = _run_command(                                                                   │
│   454 │     shell.var_expand(cmd, depth=2), clear_streamed_output=False                          │
│   455   )                                                                                        │
│   456   shell.user_ns['_exit_code'] = result.returncode                                          │
│                                                                                                  │
│ /usr/local/lib/python3.9/dist-packages/google/colab/_system_commands.py:167 in _run_command      │
│                                                                                                  │
│   164   """Calls the shell command, forwarding input received on the stdin_socket."""            │
│   165   locale_encoding = locale.getpreferredencoding()                                          │
│   166   if locale_encoding != _ENCODING:                                                         │
│ ❱ 167 │   raise NotImplementedError(                                                             │
│   168 │   │   'A UTF-8 locale is required. Got {}'.format(locale_encoding)                       │
│   169 │   )                                                                                      │
│   170                                                                                            │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
NotImplementedError: A UTF-8 locale is required. Got ANSI_X3.4-1968

In [ ]:
model = ""
trainer = ""

In [ ]:
tokenized_datasets['train'][0]['targets']

'{click, 14, , stop}'

In [ ]:
# Test compute metrics
pred = tokenized_datasets['train'][0]['labels']
reference = tokenized_datasets['train'][0]['targets']
pred_labels = [pred, pred]
print(compute_metrics(pred_labels))
decoded_pred = tokenizer.batch_decode(pred, skip_special_tokens=True)
print(decoded_pred)

result = rouge.compute(predictions=decoded_pred, references=decoded_pred, use_stemmer=True)
print(result)
prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for p in pred]
print(prediction_lens)
result["gen_len"] = np.mean(prediction_lens)
print(result)

{'rouge1': 0.0714, 'rouge2': 0.0, 'rougeL': 0.0714, 'rougeLsum': 0.0714, 'gen_len': 0.2381}
['', '', 'click', ',', '14,', '', ',', 'stop', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '']
{'rouge1': 0.07142857142857142, 'rouge2': 0.0, 'rougeL': 0.07142857142857142, 'rougeLsum': 0.07142857142857142}
[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
{'rouge1': 0.07142857142857142, 'rouge2': 0.0, 'rougeL': 0.07142857142857142, 'rougeLsum': 0.07142857142857142, 'gen_len': 1.0}


# Test Tensorflow

In [ ]:
from transformers import create_optimizer, AdamWeightDecay

optimizer = AdamWeightDecay(learning_rate=2e-5, weight_decay_rate=0.01)

In [ ]:
from transformers import TFAutoModelForSeq2SeqLM

model = TFAutoModelForSeq2SeqLM.from_pretrained(checkpoint)

All model checkpoint layers were used when initializing TFT5ForConditionalGeneration.

All the layers of TFT5ForConditionalGeneration were initialized from the model checkpoint at t5-base.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFT5ForConditionalGeneration for predictions without further training.


In [ ]:
from transformers import DataCollatorForSeq2Seq

data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=checkpoint, return_tensors="tf")

In [ ]:
tf_train_set = model.prepare_tf_dataset(
    tokenized_datasets["train"],
    shuffle=True,
    batch_size=8,
    collate_fn=data_collator,
)

tf_test_set = model.prepare_tf_dataset(
    tokenized_datasets["test"],
    shuffle=False,
    batch_size=8,
    collate_fn=data_collator,
)

tf_validation_set = model.prepare_tf_dataset(
    tokenized_datasets["validation"],
    shuffle=False,
    batch_size=8,
    collate_fn=data_collator,
)

You're using a T5TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


In [ ]:
import tensorflow as tf

model.compile(optimizer=optimizer)

No loss specified in compile() - the model's internal loss computation will be used as the loss. Don't panic - this is a common way to train TensorFlow models in Transformers! To disable this behaviour please pass a loss argument, or explicitly pass `loss=None` if you do not want your model to compute a loss.


In [ ]:
from transformers.keras_callbacks import KerasMetricCallback

metric_callback = KerasMetricCallback(metric_fn=compute_metrics, eval_dataset=tf_validation_set)

In [ ]:
from transformers.keras_callbacks import PushToHubCallback

push_to_hub_callback = PushToHubCallback(
    output_dir="tf_T5_hierarchical_1",
    tokenizer=tokenizer,
)

/content/tf_T5_hierarchical_1 is already a clone of https://huggingface.co/LucasThil/tf_T5_hierarchical_1. Make sure you pull the latest changes with `repo.git_pull()`.


In [ ]:
callbacks = [metric_callback, push_to_hub_callback]

In [ ]:
model.fit(x=tf_train_set, validation_data=tf_test_set, epochs=3, callbacks=callbacks)

Epoch 1/3
2992/2992 [==============================] - ETA: 0s - loss: 0.2824

ValueError: ignored

In [ ]:
model = ""

In [ ]:
optimizer = ""

# Inference

In [ ]:
text = "Devise a plan for the following instruction: Select 12rh2X4, Hjfr, iMDDWUP, Jc0Wr4A, Rg and click Submit.	"

In [ ]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("/content/T5_base_hierarchy6_256/checkpoint-4500")
inputs = tokenizer(text, return_tensors="pt").input_ids
print(inputs)

tensor([[24434,     7,    15,     3,     9,   515,    21,     8,   826,  8033,
            10,  6185,   586,    52,   107,   357,     4,  8525,   454,   354,
            89,    52,     6,     3,    23, 11731, 20293,  6880,     6,   446,
            75,   632,   518,    52,   591,   188,     6,   391,   122,    11,
          1214,  3325,  1538,     5,     1]])


In [ ]:
from transformers import AutoModelForSeq2SeqLM

model = AutoModelForSeq2SeqLM.from_pretrained("/content/T5_base_hierarchy6_256/checkpoint-4500")
outputs = model.generate(inputs, max_new_tokens=100, do_sample=False)
print(outputs)

tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0]])


In [ ]:
tokenizer.decode(outputs[0], skip_special_tokens=True)

''